# Wikipedia2014 + Gigaword5 and GloVe
ref: https://nlp.stanford.edu/projects/glove/

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

from keras.datasets import reuters
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, LSTM, Flatten, Dense, Dropout, Reshape, Bidirectional, Conv2D, MaxPool2D, Concatenate, Activation
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 300
VALIDATION_SPLIT = 0.3
#############################################################################
# TODO: this should be the same as the dimension of word embedding you are  #
# using                                                                     #
#############################################################################
EMBEDDING_DIM = None
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################


from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from sklearn.decomposition import PCA

In [ ]:
#############################################################################
# TODO: please assign the absolute path to the GloVe txt file               #
# (glove.6B.50d.txt or any dimension you like)                              #
#############################################################################
path_to_glove = None
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################
tmp_file_name = 'word2vec.6B.txt'

glove_file = datapath(path_to_glove)
tmp_file = get_tmpfile(tmp_file_name)

glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

Print the type the word vector of 'animal', the dimension (shape) of this vector, and the content of thie vector.

If you use 300 dimensions word vectors, the correct outout will be something like this:
```
(numpy.ndarray,
 (300,),
 array([ 0.25653  ,  0.66592  , -0.5313   ,  0.20342  ,  0.40049  ,
 ... multiple lines are omitted ...
        -0.14921  ,  0.2404   ,  0.22182  ,  0.68883  , -0.018991 ],
       dtype=float32))
```

In [ ]:
#############################################################################
# TODO: please print the type the word vector of 'animal', the dimension    #
# (shape) of this vector, and the content of thie vector.                   #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

Use [similar_by_vector](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.similar_by_vector) to obtain the 10 most similar word to 'france'

If you use 300 dimensions word vectors, the correct outout will be something like this:
```
[('france', 1.0),
 ('french', 0.7344760894775391),
 ('paris', 0.6580672264099121),
 ('belgium', 0.620672345161438),
 ('spain', 0.573593258857727),
 ('italy', 0.5643459558486938),
 ('germany', 0.5567397475242615),
 ('prohertrib', 0.5564222931861877),
 ('britain', 0.5553334951400757),
 ('chirac', 0.5362644195556641)]
```

In [ ]:
#############################################################################
# TODO: use similar_by_vector to obtain the 10 most similar word to 'france'#
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

Use [most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.Word2VecKeyedVectors.most_similar) to obtain the result of 'woman' + 'king' - 'man'

If you use 300 dimensions, the correct output will be somthing like this:
```
[('queen', 0.6713277101516724),
 ('princess', 0.5432624220848083),
 ('throne', 0.5386104583740234),
 ('monarch', 0.5347574949264526),
 ('daughter', 0.498025119304657),
 ('mother', 0.4956442713737488),
 ('elizabeth', 0.4832652509212494),
 ('kingdom', 0.47747087478637695),
 ('prince', 0.4668239951133728),
 ('wife', 0.4647327661514282)]
```

In [ ]:
#############################################################################
# TODO: use most_similar to obtain the result of 'woman' + 'king' - 'man'   #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

Use [doesnt_match](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.doesnt_match) to obtain the word with the different meaning with the other words in a list 'breakfast', 'lunch', 'dinner', 'cereal'

If you use 300 dimensions, the correct output will be something like this:
```
'cereal'
```

In [ ]:
#############################################################################
# TODO: Use doesnt_match to obtain the word with the different meaning with #
# the other words in a list 'breakfast', 'lunch', 'dinner', 'cereal'        #
#############################################################################

#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
def pca_plot(model):
    labels = []
    tokens = []

    for word in model.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = PCA(n_components = 2)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    for i in range(len(x)):
        label = labels[i]
        if label in ['king', 'queen', 'sister', 'brother', 'niece', 'nephew', 'aunt', 'uncle', 'woman', 'man', 
                     'madam', 'sir']:
            plt.scatter(x[i],y[i])
            plt.annotate(label,
                         xy = (x[i], y[i]),
                         xytext = (5, 2),
                         textcoords = 'offset points',
                         ha = 'right',
                         va = 'bottom')
    plt.show()

pca_plot(model)

# Reuter-21578 Data set with word2vec
Last week, you have already worked on Reuters-21578 dataset for multi-class classification. This week, you are using word2vec to classify the same dataset.

In this lab, you will have to implement 3 three neural network models using keras API:
1. Mutilayer perceptron
2. Conv1D
3. LSTM or GRU

In [ ]:
# this requires download from the first time
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = MAX_NB_WORDS, 
                                                         skip_top = 0, 
                                                         maxlen = MAX_SEQUENCE_LENGTH,
                                                         seed = 113,
                                                         start_char = 1, 
                                                         oov_char = 2, 
                                                         index_from = 3)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

word2index = reuters.get_word_index()
word2index = {key : (value + 3) for (key, value) in word2index.items()}
word2index['<PAD>'] = 0
word2index['<START>'] = 1
word2index['<UNK>'] = 2

In [ ]:
#############################################################################
# TODO: create a embeddings_index to map word to word vector and print the  #
# number of words in the embedding                                          #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################


#############################################################################
# TODO: create a embedding_matrix that map the index of word2index to word  #
# vectors                                                                   #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

#############################################################################
# TODO: define an embedding layer that initialize the weights with          #                                         
# embedding_matrix and set trainable to False                               #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################
embedding_layer = Embedding(len(word2index) + 1,
                            EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length = MAX_SEQUENCE_LENGTH,
                            trainable = False)

In [ ]:
#############################################################################
# TODO: Use keras pad_sequences to pad the sequence in X_train and X_test   #
# to MAX_SEQUENCE_LENGTH                                                    #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
print('Shape of X_train tensor:', X_train.shape)
print('Shape of X_test tensor:', X_test.shape)

# split the data into a training set and a validation set
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

X_val = X_train[-nb_validation_samples:]
y_val = y_train[-nb_validation_samples:]
X_train = X_train[:-nb_validation_samples]
y_train = y_train[:-nb_validation_samples]

## word2vec + multiple layer perceptron
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          9294900   
_________________________________________________________________
flatten_1 (Flatten)          (None, 90000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               11520128  
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                1518      
_________________________________________________________________
activation_4 (Activation)    (None, 46)                0         
=================================================================
Total params: 20,826,882
Trainable params: 11,531,982
Non-trainable params: 9,294,900
_________________________________________________________________
```

In [ ]:
#############################################################################
# TODO: Define the model of multiple layer perceptron similar to the model  #
# summary above into the variable model, compile it and fit it.             #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
results = model.evaluate(X_test, y_test)
results

## word2vec + Conv1D
```
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          9294900   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 296, 64)           96064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3776)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3776)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                241728    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 46)                2990      
=================================================================
Total params: 9,635,682
Trainable params: 340,782
Non-trainable params: 9,294,900
_________________________________________________________________
```

In [ ]:
#############################################################################
# TODO: Define the model of 1D convolution similar to the model summary     #
# above into the variable model, compile it and fit it.                     #
#############################################################################
pass
#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

In [ ]:
results = model.evaluate(X_test, y_test)
results

## word2vec + LSTM (GRU)
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_3 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          9294900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_7 (Dense)              (None, 46)                5934      
=================================================================
Total params: 9,520,482
Trainable params: 225,582
Non-trainable params: 9,294,900
_________________________________________________________________
```

In [ ]:
sequence_input = Input(shape = (MAX_SEQUENCE_LENGTH,), dtype = 'int32')
embedded_sequences = embedding_layer(sequence_input)

units = 128
embedded_sequences = embedding_layer(sequence_input)
layer1 = LSTM(units,
    dropout = 0.2,
    recurrent_dropout = 0.2,
    return_sequences = False)
x = layer1(embedded_sequences)

final_layer = Dense(46, activation = 'softmax')
preds = final_layer(x)

model = Model(sequence_input, preds)
print(model.summary())
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['acc'])

model.fit(X_train, y_train, validation_data = (X_val, y_val),
          epochs = 20, batch_size = 256)

In [ ]:
results = model.evaluate(X_test, y_test)
results